In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.kernel_approximation import Nystroem
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

# Load the Breast Cancer dataset
data = load_breast_cancer()
X, y = data.data, data.target

In [2]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
rf_acc = accuracy_score(y_test, y_pred_rf)
print("Random Forest Accuracy:", rf_acc)
print(classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.9649122807017544
              precision    recall  f1-score   support

           0       0.98      0.93      0.95        43
           1       0.96      0.99      0.97        71

    accuracy                           0.96       114
   macro avg       0.97      0.96      0.96       114
weighted avg       0.97      0.96      0.96       114



In [3]:
# Apply Nyström method to approximate the RBF kernel
nystroem = Nystroem(kernel='rbf', gamma=0.1, n_components=300, random_state=42)
svm = SVC(kernel='linear', random_state=42)

# Create a pipeline for SVM with the Nyström method
svm_nystroem = Pipeline([
    ('feature_map', nystroem),
    ('svm', svm)
])

# Train the SVM with Nyström approximation
svm_nystroem.fit(X_train, y_train)
y_pred_svm = svm_nystroem.predict(X_test)
svm_acc = accuracy_score(y_test, y_pred_svm)
print("SVM with Nyström Kernel Accuracy:", svm_acc)
print(classification_report(y_test, y_pred_svm))


SVM with Nyström Kernel Accuracy: 0.6228070175438597
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        43
           1       0.62      1.00      0.77        71

    accuracy                           0.62       114
   macro avg       0.31      0.50      0.38       114
weighted avg       0.39      0.62      0.48       114



c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [5]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import Aer
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms import QSVC

In [6]:
def quantum_feature_map(x):
    n_qubits = len(x)
    qc = QuantumCircuit(n_qubits)
    for i, xi in enumerate(x):
        qc.ry(xi, i)
    qc.barrier()
    for i in range(n_qubits):
        for j in range(i+1, n_qubits):
            qc.cx(i, j)
            qc.rz(x[i] * x[j], j)
            qc.cx(i, j)
    return qc

class QuantumDecisionTreeNode:
    def __init__(self, depth=0, max_depth=3):
        self.left = None
        self.right = None
        self.feature = None
        self.threshold = None
        self.depth = depth
        self.max_depth = max_depth
        self.leaf_value = None

    def fit(self, X, y):
        if self.depth == self.max_depth or len(np.unique(y)) == 1:
            self.leaf_value = np.argmax(np.bincount(y))
            return

        n_features = X.shape[1]
        best_gini = float('inf')
        best_feature = None
        best_threshold = None

        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask
                gini = self.gini_impurity(y[left_mask]) + self.gini_impurity(y[right_mask])
                if gini < best_gini:
                    best_gini = gini
                    best_feature = feature
                    best_threshold = threshold

        self.feature = best_feature
        self.threshold = best_threshold

        left_mask = X[:, self.feature] <= self.threshold
        right_mask = ~left_mask

        self.left = QuantumDecisionTreeNode(depth=self.depth+1, max_depth=self.max_depth)
        self.right = QuantumDecisionTreeNode(depth=self.depth+1, max_depth=self.max_depth)

        self.left.fit(X[left_mask], y[left_mask])
        self.right.fit(X[right_mask], y[right_mask])

    def predict(self, x):
        if self.leaf_value is not None:
            return self.leaf_value

        if x[self.feature] <= self.threshold:
            return self.left.predict(x)
        else:
            return self.right.predict(x)

    @staticmethod
    def gini_impurity(y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return 1 - np.sum(probabilities**2)


class QuantumDecisionTree:
    def __init__(self, max_depth=3):
        self.root = QuantumDecisionTreeNode(max_depth=max_depth)

    def fit(self, X, y):
        quantum_X = np.array([self.quantum_feature_extraction(x) for x in X])
        self.root.fit(quantum_X, y)

    def predict(self, X):
        return np.array([self.root.predict(self.quantum_feature_extraction(x)) for x in X])

    @staticmethod
    def quantum_feature_extraction(x):
        qc = quantum_feature_map(x)
        backend = Aer.get_backend('aer_simulator')
        qc.save_statevector()
        transpiled_circuit = transpile(qc, backend)
        job = backend.run(transpiled_circuit)
        result = job.result()
        statevector = result.get_statevector()
        return np.real(statevector)


In [9]:
from sklearn.decomposition import PCA 

pca = PCA(n_components=5)
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)

qdt = QuantumDecisionTree(max_depth=3)
qdt.fit(X_train_reduced, y_train)
y_pred_qdt = qdt.predict(X_test_reduced)
qdt_acc = accuracy_score(y_test, y_pred_qdt)
print("Quantum Decision Tree Accuracy:", qdt_acc)
print(classification_report(y_test, y_pred_qdt))


Quantum Decision Tree Accuracy: 0.6403508771929824
              precision    recall  f1-score   support

           0       0.75      0.07      0.13        43
           1       0.64      0.99      0.77        71

    accuracy                           0.64       114
   macro avg       0.69      0.53      0.45       114
weighted avg       0.68      0.64      0.53       114



In [10]:
from qiskit.visualization import circuit_drawer

def save_quantum_circuit(x, filename='quantum_circuit.png'):
    qc = quantum_feature_map(x)
    fig = circuit_drawer(qc, output='mpl')
    fig.savefig(filename)
    print(f"Circuit saved as '{filename}'")

sample_x = X_train[0]
save_quantum_circuit(sample_x)

import matplotlib.pyplot as plt
import cv2

image_path = "quantum_circuit.png"
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.imshow(image)
plt.axis("off")
plt.show()

Circuit saved as 'quantum_circuit.png'
